# How to train the yolov8 model with Azure Machine learning - Python SDK
Azure Machine Learning provides a comprehensive solution for managing the entire lifecycle of machine learning models. In this tutorial, we'll explore how to use AzureML to train and continuously improve an open source model. Here we will train the [Yolov8 model](https://github.com/ultralytics/ultralytics) object detection model developed by Ultralytics.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Let's login configure your workspace and resource group.
credential = DefaultAzureCredential()

# Get a handle to the workspace. You can find the info on the workspace tab on ml.azure.com
ml_client = MLClient(
    credential=credential,
    subscription_id="<your-subscription-id>",
    resource_group_name="<your-resource-group-name>",
    workspace_name="your-azureml-workspace-name",
)

## Create an AzureML environment
We need to create an [AzureML environment](https://learn.microsoft.com/azure/machine-learning/how-to-manage-environments-v2) with all the required dependencies to run our training.

We will create a folder `azureml-environment`, it will contain the docker-context to build the environment. In this folder, we will add a Dockerfile with the required dependencies to run our training:


In [ ]:

%%bash
mkdir azureml-environment
echo """
FROM pytorch/pytorch:2.0.0-cuda11.7-cudnn8-runtime

# Downloads to user config dir
ADD https://ultralytics.com/assets/Arial.ttf https://ultralytics.com/assets/Arial.Unicode.ttf /root/.config/Ultralytics/

# Install linux packages
ENV DEBIAN_FRONTEND noninteractive
RUN apt update
RUN TZ=Etc/UTC apt install -y tzdata
RUN apt install --no-install-recommends -y gcc git zip curl htop libgl1-mesa-glx libglib2.0-0 libpython3-dev gnupg g++

# Security updates
# https://security.snyk.io/vuln/SNYK-UBUNTU1804-OPENSSL-3314796
RUN apt upgrade --no-install-recommends -y openssl tar

RUN pip install ultralytics==8.0.180
RUN pip install azureml-mlflow==1.52.0
RUN pip install mlflow==2.4.2
""" > azureml-environment/Dockerfile


Note that Ultralytics provides [Dockerfiles for different platform](https://github.com/ultralytics/ultralytics/tree/main/docker). Here we used the same base image and installed the same linux dependencies than the [amd64 Dockerfile](https://github.com/ultralytics/ultralytics/blob/main/docker/Dockerfile), but we installed the ultralytics package with pip install to control the version we install and make sure the package version is deterministic. To track hyperparameters and metrics in AzureML, we installed [mlflow](https://pypi.org/project/mlflow/) and [azureml-mlflow](https://pypi.org/project/azureml-mlflow/). This enables us to evaluate our model performance easily and compare models from various training runs in AzureML studio.
Now let's create our environment:

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext

env_docker_context = Environment(
    build=BuildContext(path="azureml-environment"),
    name="yolov8-environment",
    description="Environment created from a Docker context.",
)
ml_client.environments.create_or_update(env_docker_context)

## Create an AzureML compute cluster
We need a compute instance from where we can run the training. We will create a [compute cluster](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?view=azureml-api-2&tabs=python#what-is-a-compute-cluster) that auto-scales from 0 to 2 active nodes.
We will make sure that:

- Idling nodes scale down after 2 minutes of inactivity
- The minimum number of running nodes is 0, to avoid the cost of idling nodes.

You can find [more information about clusters auto-scaling here](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-optimize-cost?view=azureml-api-2#configure-training-clusters-for-autoscaling).

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster = AmlCompute(
    name="cluster-with-1-k80-gpu",
    type="amlcompute",
    size="Standard_NC6",
    location="westeurope",
    min_instances=0,
    max_instances=2,
    idle_time_before_scale_down=120,
)
ml_client.begin_create_or_update(cluster).result()

## Create an AzureML dataset
For this tutorial we will use the [coco128 dataset](https://www.kaggle.com/ultralytics/coco128). We will create an [AzureML data asset](https://learn.microsoft.com/azure/machine-learning/how-to-create-data-assets) to bookmark our dataset and easily use the dataset for various trainings.

Let's download our training dataset:

In [ ]:
%%bash
# For this tutorial we will use the coco128 dataset: https://www.kaggle.com/ultralytics/coco128. We will create an AzureML data asset to bookmark our dataset and easily use the dataset for various trainings.
wget https://ultralytics.com/assets/coco128.zip
unzip coco128.zip

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Create AzureML dataset

my_data = Data(
    path="coco128",
    type=AssetTypes.URI_FOLDER,
    description="Coco 128 dataset",
    name="coco128"
)

ml_client.data.create_or_update(my_data)

Here we specified the local path of the dataset, which means that the dataset will be uploaded from your local to AzureML. But note that AzureML dataset supports [several type of paths](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?view=azureml-api-2&tabs=cli#supported-paths), for example a path on Azure storage. Your local dataset will be uploaded to AzureML. Now your dataset name should be `azureml:coco128:1`. You can see your dataset in AzureML studio in Data > Data asset. Note that if you create a dataset with the same name several time, it will create several versions of your dataset.

## Register a pre-trained model
We will train a pre-trained model. Note that you can find the [yolov8 pre-trained models here](https://github.com/ultralytics/ultralytics/blob/main/README.md#models).

Let's download the yolov8n.pt model:

In [ ]:
%%bash
# Download the yolov8 model
wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

In [ ]:
# Register model
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="yolov8n.pt",
    type=AssetTypes.CUSTOM_MODEL,
    name="yolov8n",
    description="yolov8n model.",
)
ml_client.models.create_or_update(file_model)

## Training code
We will create a `training-code` folder containing the required files to run our training.
I want to show you how you can create your custom dataset definition. So we will download the [coco128.yaml](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/datasets/coco128.yaml) and call it custom-coco128.yaml.
We have to put this file in the `training-code` folder to make sure it is available when running our training:

In [ ]:
%%bash
mkdir training-code
wget https://raw.githubusercontent.com/ultralytics/ultralytics/7f37790134527e328c475ac292efdc39b1cd7b5e/ultralytics/cfg/datasets/coco128.yaml -O training-code/custom-coco128.yaml

We want to ensure that the AzureML job uses our dataset coco128, rather than downloading it during the job, so let's remove the last line of the `custom-coco128.yaml`:

In [ ]:
%%bash
sed -i "s|download:.*$||" training-code/custom-coco128.yaml

## Run the training

An AzureML job execute a task against a compute target. We will create an AzureMl job that executes the yolov8 training against the compute cluster we created earlier.

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input

job = command(
    inputs=dict(
        training_data=Input(
            type="uri_folder",
            path="azureml:coco128:1",
        ),
        model_to_train=Input(
            type="custom_model",
            path="azureml:yolov8n:1"
        )
    ),
    code="training-code",
    command="""
        sed -i "s|path:.*$|path: ${{ inputs.training_data }}|" custom-coco128.yaml &&
        yolo task=detect train data=custom-coco128.yaml model=${{ inputs.model_to_train }} epochs=3 project=yolov8-experiment name=experiment
    """,
    environment="azureml:yolov8-environment:1",
    compute="cluster-with-1-k80-gpu",
    display_name="yolov8-experiment",
    experiment_name="yolov8-experiment"
)

Let's have a closer look to this job definition.

You can see that we defined an input `training_data`, this is our coco dataset.
AzureML will mount or download this dataset, and when using `${{ inputs.training_data }}` in the command, AzureML will take care of resolving the filesystem path value.

We will train our model with the following command:

```bash
yolo task=detect train data=custom-coco128.yaml model=${{ inputs.model_to_train }} epochs=3 project=yolov8-experiment name=experiment
```

Here we hard-coded `epochs=3`. This value could be passed as an input parameter.
You can look at the [ultralytics documentation](https://github.com/ultralytics/ultralytics/blob/main/docs/usage/cfg.md#train) to get more details about each settings.

In this command we pass `data=custom-coco128.yaml`. Our dataset definition `custom-coco128.yaml` should contain a setting called path, that represents the dataset root dir. For now its value is:

```yaml
path: ../datasets/coco128
```

We want to change the path value to be the path of our AzureML dataset coco128.  
In AzureML jobs, [job inputs datasets can be accessed by mounting or downloading](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-job-command?view=azureml-api-2#job-inputs) them. When you use ${{ inputs.training_data }} in a command, AzureML resolves the filesystem path of the dataset. However, it is not guaranteed that the filesystem path is consistent between different job runs, so we can't hardcode the path in the yaml file.
To work around this, you can dynamically edit the path in the AzureML job, just before running the training.
That's why we added the following sed command to replace `path: <anything>` by `path: <path-to-our-training-dataset>`.

```bash
sed -i "s|path:.*$|path: ${{ inputs.training_data }}|" coco128.yaml
```

We also defined a `model_to_train`. This is the pre-trained model that we will start the training from.

Now let's submit the job:

In [ ]:
ml_client.create_or_update(job)